### testing the importance of distance on instruction priority

In [ ]:
%pip install huggingface_hub
%pip install python-dotenv
%pip install huggingface
%pip install datasets
%pip install transformers

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
huggingface_token = os.getenv("HUGGINGFACE_TOKEN")

In [ ]:
!hf auth login --token {huggingface_token}

In [ ]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
#load wiki
ds = load_dataset("wikitext", "wikitext-2-raw-v1")

In [ ]:
print("hi")